## Preprocessing

In [1]:
#  Install keras-tuner
! pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.1 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras_tuner as kt


In [3]:
#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN"], axis=1)
application_df.sample(10)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
7629,OUR FATHERS SLE,T3,Independent,C1000,Preservation,Trust,1,25000-99999,N,44889,1
29978,LITTLE LEAGUE BASEBALL INC,T3,CompanySponsored,C1000,Preservation,Trust,1,100000-499999,N,71615,0
26433,CISD EDUCATIONAL SUPPORT GROUPS INC,T3,CompanySponsored,C1000,Preservation,Trust,1,0,N,5000,0
23615,FRIENDS OF ALLIED NON-PROFITS,T3,Independent,C1000,Preservation,Trust,1,25000-99999,N,2119667,0
14529,VILLAGE ASHANTI,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
16915,PARENT BOOSTER USA INC,T3,CompanySponsored,C2100,Preservation,Trust,1,0,N,5000,0
15207,UNITED INC,T3,Independent,C7000,Preservation,Trust,1,100000-499999,N,69553,1
17265,CAL POLY CORPORATION VEBA TR,T9,Independent,C1000,ProductDev,Co-operative,1,5M-10M,N,23572470,0
10375,ARIZONA CHAP OF COLLEGE AND UNIV PRO ASSOC FOR...,T6,Independent,C1000,ProductDev,Trust,1,0,N,5000,0
27663,ASSOCIATION OF CALIFORNIA UNIVERSITY PROFESSORS,T3,Independent,C2000,Preservation,Trust,1,0,N,5000,1


In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at NAME value counts for binning
name_counts = application_df['NAME'].value_counts()
name_counts

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [10]:
# Choose a cutoff value and create a list of name to be replaced
# use the variable name `name_to_replace`
name_to_replace = list(name_counts[name_counts < 400].index)

# Replace in dataframe
for name in name_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                              30674
PARENT BOOSTER USA INC                              1260
TOPS CLUB INC                                        765
UNITED STATES BOWLING CONGRESS INC                   700
WASHINGTON STATE UNIVERSITY                          492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC      408
Name: NAME, dtype: int64

In [12]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [13]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_1 = class_counts.loc[class_counts > 1]
class_counts_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [14]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [15]:
# Look at ASK_AMT value counts for binning
ask_amt_counts = application_df['ASK_AMT'].value_counts()
ask_amt_counts

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [16]:
# You may find it helpful to look at ASK_AMT value counts >1
ask_amt_1 = ask_amt_counts.loc[ask_amt_counts > 1]
ask_amt_1

5000      25398
10478         3
15583         3
63981         3
6725          3
          ...  
100000        2
8181          2
24746         2
45793         2
240780        2
Name: ASK_AMT, Length: 152, dtype: int64

In [17]:
# Choose a cutoff value and create a list of ask_amt to be replaced
# use the variable name `ask_amt_to_replace`
ask_amt_to_replace = list(ask_amt_counts[ask_amt_counts < 3].index)

# Replace in dataframe
for amt in ask_amt_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,"Other")

# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

5000     25398
Other     8889
15583        3
6725         3
10478        3
63981        3
Name: ASK_AMT, dtype: int64

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
app_final = pd.get_dummies(application_df)
app_final.sample(10)

,STATUS,IS_SUCCESSFUL,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_Other,NAME_PARENT BOOSTER USA INC,NAME_TOPS CLUB INC,NAME_UNITED STATES BOWLING CONGRESS INC,NAME_WASHINGTON STATE UNIVERSITY,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,...,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_6725,ASK_AMT_10478,ASK_AMT_15583,ASK_AMT_63981,ASK_AMT_Other
24977,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
31889,1,0,0,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
18029,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
11778,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
4652,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
17021,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
17463,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
34216,1,1,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
16300,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
22046,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [19]:
# Split our preprocessed data into our features and target arrays
y = app_final["IS_SUCCESSFUL"]
X = app_final.drop(["IS_SUCCESSFUL"], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size = 0.7)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model using kerastuner

In [21]:
# Create a method that creates a new Sequential model with hyperparameter options
number_input_features = len(X_train.values[0])
def create_model(hp):
  model = Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
  activation_choice = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
  model.add(Dense(units=hp.Int('first_units', min_value=1, max_value=50, step=5), activation=activation_choice, input_dim = number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
  for i in range(hp.Int("num_layers", 1, 5)):
    model.add(Dense(units=hp.Int(f'units_{i}', min_value=1, max_value=50, step=5), activation=activation_choice))
  model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
  model.compile(loss="binary_crossentropy", optimizer='Adam', metrics=["accuracy"])
  return model

In [22]:
# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(create_model, objective='val_accuracy', max_epochs=50, hyperband_iterations=3)

In [23]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))

Trial 270 Complete [00h 02m 24s]
val_accuracy: 0.7291545271873474

Best val_accuracy So Far: 0.7327502369880676
Total elapsed time: 02h 09m 26s


In [24]:
# Get top 3 model hyperparameters and print the values
for params in tuner.get_best_hyperparameters(3):
  print(params.values)

{'activation': 'relu', 'first_units': 21, 'num_layers': 3, 'units_0': 11, 'units_1': 11, 'units_2': 1, 'units_3': 1, 'tuner/epochs': 50, 'tuner/initial_epoch': 17, 'tuner/bracket': 3, 'tuner/round': 3, 'tuner/trial_id': '0046', 'units_4': 26}
{'activation': 'relu', 'first_units': 11, 'num_layers': 1, 'units_0': 16, 'units_1': 31, 'units_2': 41, 'units_3': 6, 'units_4': 11, 'tuner/epochs': 6, 'tuner/initial_epoch': 2, 'tuner/bracket': 3, 'tuner/round': 1, 'tuner/trial_id': '0191'}
{'activation': 'relu', 'first_units': 41, 'num_layers': 3, 'units_0': 46, 'units_1': 46, 'units_2': 46, 'units_3': 26, 'units_4': 6, 'tuner/epochs': 17, 'tuner/initial_epoch': 6, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0238'}


In [25]:
# Get best model hyperparameters
best_model = tuner.get_best_hyperparameters(1)[0]
best_model.values

{'activation': 'relu',
 'first_units': 21,
 'num_layers': 3,
 'units_0': 11,
 'units_1': 11,
 'units_2': 1,
 'units_3': 1,
 'tuner/epochs': 50,
 'tuner/initial_epoch': 17,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0046',
 'units_4': 26}

In [26]:
# Evaluate the top 3 models against the test dataset
for model in tuner.get_best_models(3):
  model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 1s - loss: 0.5327 - accuracy: 0.7328 - 590ms/epoch - 2ms/step
322/322 - 1s - loss: 0.5420 - accuracy: 0.7325 - 561ms/epoch - 2ms/step
322/322 - 1s - loss: 0.5342 - accuracy: 0.7325 - 594ms/epoch - 2ms/step
Loss: 0.5341612696647644, Accuracy: 0.7324587106704712


In [27]:
# Export our model to HDF5 file
model.save("AlphabetSoupCharity_Optimization.h5")
